In [57]:
import pandas as pd
import sklearn
import wordcloud
from collections import Counter
from nltk.sentiment import SentimentAnalyzer

In [2]:
df = pd.read_csv("reddit_vm.csv")

In [3]:
df.head()

,title,score,id,url,comms_num,created,body,timestamp
0,Health Canada approves AstraZeneca COVID-19 va...,7,lt74vw,https://www.canadaforums.ca/2021/02/health-can...,0,1.614400e+09,NaN,2021-02-27 06:33:45
1,COVID-19 in Canada: 'Vaccination passports' a ...,2,lsh0ij,https://www.canadaforums.ca/2021/02/covid-19-i...,1,1.614316e+09,NaN,2021-02-26 07:11:07
2,Coronavirus variants could fuel Canada's third...,6,lohlle,https://www.canadaforums.ca/2021/02/coronaviru...,0,1.613887e+09,NaN,2021-02-21 07:50:08
3,Canadian government to extend COVID-19 emergen...,1,lnptv8,https://www.canadaforums.ca/2021/02/canadian-g...,0,1.613796e+09,NaN,2021-02-20 06:35:13
4,Canada: Pfizer is 'extremely committed' to mee...,6,lkslm6,https://www.canadaforums.ca/2021/02/canada-pfi...,0,1.613468e+09,NaN,2021-02-16 11:36:28


In [5]:
df.shape

(1514, 8)

In [6]:
df.columns

Index(['title', 'score', 'id', 'url', 'comms_num', 'created', 'body',
       'timestamp'],
      dtype='object')

 sentiment analysis - 
1. url, id, date of creation, timestamp doesnt matter
2. comms_num and score are not independent 
3. Unsupervised problem
4. is there a way to identify comments - yes using "comment" in title
5. all comments have a body, we can replace nulls in body through comment
6. how many distinct sites are there. can we bucket them
7. 70 percent of data is comments

In [8]:
df.isna().sum()

title           0
score           0
id              0
url          1062
comms_num       0
created         0
body          366
timestamp       0
dtype: int64

In [9]:
df[df['body'].isna()]

,title,score,id,url,comms_num,created,body,timestamp
0,Health Canada approves AstraZeneca COVID-19 va...,7,lt74vw,https://www.canadaforums.ca/2021/02/health-can...,0,1.614400e+09,NaN,2021-02-27 06:33:45
1,COVID-19 in Canada: 'Vaccination passports' a ...,2,lsh0ij,https://www.canadaforums.ca/2021/02/covid-19-i...,1,1.614316e+09,NaN,2021-02-26 07:11:07
2,Coronavirus variants could fuel Canada's third...,6,lohlle,https://www.canadaforums.ca/2021/02/coronaviru...,0,1.613887e+09,NaN,2021-02-21 07:50:08
3,Canadian government to extend COVID-19 emergen...,1,lnptv8,https://www.canadaforums.ca/2021/02/canadian-g...,0,1.613796e+09,NaN,2021-02-20 06:35:13
4,Canada: Pfizer is 'extremely committed' to mee...,6,lkslm6,https://www.canadaforums.ca/2021/02/canada-pfi...,0,1.613468e+09,NaN,2021-02-16 11:36:28
...,...,...,...,...,...,...,...,...
531,I am a father of five unvaccinated children. A...,1,1v3c8o,http://www.np.reddit.com/r/conspiracy/comments...,1,1.389625e+09,NaN,2014-01-13 16:55:48
532,Love Them. Protect Them. Never Inject Them.,1,1v3c3i,http://i.imgur.com/SkB0a6T.jpg,2,1.389625e+09,NaN,2014-01-13 16:53:07
533,Vaccines Are Just Asping For Trouble,3,1v3brt,http://i.imgur.com/bIPQpbv.jpg,1,1.389624e+09,NaN,2014-01-13 16:47:31
534,Dr. Harper explained in her presentation that ...,1,1v3blj,http://www.feelguide.com/2013/07/16/lead-devel...,1,1.389624e+09,NaN,2014-01-13 16:44:57


In [10]:
df['score'].value_counts()

 1       615
 2       214
 0       139
 3       134
 4        64
 5        59
 6        50
 7        33
 8        24
-1        20
 10       18
 9        17
 11       15
-2        12
 12       11
 14       11
 13        9
 16        8
 15        7
 19        6
-4         5
-3         5
 17        5
 20        4
-5         4
 24        3
-6         3
 23        2
 25        2
 30        2
 21        2
 18        1
 43        1
-11        1
 1184      1
 31        1
 39        1
 34        1
 27        1
 26        1
 22        1
-9         1
Name: score, dtype: int64

In [13]:
df['comms_num'].value_counts()

0      1105
1       143
2        75
4        39
3        37
5        22
6        10
7         9
8         9
11        8
9         7
10        4
12        4
15        4
32        3
16        3
22        3
20        3
13        3
17        3
18        2
24        2
49        2
166       1
14        1
19        1
21        1
25        1
27        1
30        1
31        1
34        1
39        1
46        1
55        1
74        1
596       1
Name: comms_num, dtype: int64

In [17]:
df['title'].value_counts()

Comment                                                                                                                                                                                                                                                              1062
look into the false allegations made against Dr. Wakefield and the pressure put onto the medical board via the vaccine manufacturers which led to his license being revoked.. further.. a new study published in december 2013 verifies Dr. Wakefields findings..       1
Vaccines exposed                                                                                                                                                                                                                                                        1
Irrefutable Prufe                                                                                                                                                                                         

In [22]:
df[df['title']=='Comment']['body'].isna().value_counts()

False    1062
Name: body, dtype: int64

In [30]:
df['body'].fillna(df['title'], inplace=True)

In [31]:
df['body'].isna().value_counts()

False    1514
Name: body, dtype: int64

In [50]:
url_list=[]
for i in df[~df['url'].isna()]['url'].tolist():
    split_url = i.split('/')
    url_list+= split_url

In [54]:
Counter(url_list).most_common()

[('', 636),
 ('http:', 273),
 ('r', 253),
 ('comments', 252),
 ('https:', 177),
 ('www.np.reddit.com', 121),
 ('www.reddit.com', 93),
 ('VaccineMyths', 93),
 ('conspiracy', 66),
 ('np.reddit.com', 36),
 ('i.imgur.com', 35),
 ('todayilearned', 28),
 ('i.redd.it', 25),
 ('1vewtu', 24),
 ('til_the_doctor_who_claimed_there_was_a_link', 24),
 ('news', 15),
 ('2014', 14),
 ('imgur.com', 12),
 ('02', 10),
 ('Health', 10),
 ('worldnews', 9),
 ('01', 8),
 ('2021', 7),
 ('www.canadaforums.ca', 6),
 ('youtu.be', 6),
 ('08', 5),
 ('2015', 5),
 ('04', 5),
 ('article', 5),
 ('a', 5),
 ('www.naturalnews.com', 5),
 ('11', 4),
 ('25', 4),
 ('www.facebook.com', 4),
 ('changemyview', 4),
 ('www.youtube.com', 4),
 ('AskReddit', 4),
 ('health', 3),
 ('VaccineDiscussion', 3),
 ('vaxopedia.org', 3),
 ('2017', 3),
 ('05', 3),
 ('03', 3),
 ('articles', 3),
 ('archive', 3),
 ('06', 3),
 ('m.imgur.com', 3),
 ('2013', 3),
 ('abcnews.go.com', 2),
 ('posts', 2),
 ('2018', 2),
 ('07', 2),
 ('21', 2),
 ('10', 2),
 ('

In [56]:
# df.to_csv("vm_clean.csv",index=False)